# Bank Heist Project Submission

**Description**

Given a list of bank, their locations, the amount of money each bank holds, and the amount of time it would take to rob each bank; apply your knowledge of algorithms to make as much profit as possible!

**Design a solution that finds a list of banks that when robbed will maximize your profit, while following these constraints:**

- Total time for robbery cannot exceed **24 hours**. This includes travel time between banks + time it takes to rob the bank + time it takes to get to the helicopter escape zone

- Travel speed from one bank to another is **30 km/h**
    - Use the x and y coordinates to calculate the distance (in km) between banks
    - Use the distance and travel speed to calculate the amount of time it takes to get from one bank to another

- Solution must run under **3 minutes**

- You can start anywhere, but you have to end at the **helicopter escape zone** located at coordinates **(0,0)**

**Hints**

Most of the design paradigms you saw in class will work for this. Start with something that's easier (brute-force or greedy algorithm) and then work towards a better design once it works:

    - Divide-and-conquer
    - Brute Force
    - Greedy Algorithm
    - Dynamic Programming
    - Backtracking
    - Breadth-first & Depth-first search
Some we haven't covered:

    - Branch & Bound
    - Prune & Search
    
Because there are too many banks at each step, you will need to select only some candidates to explore.

If you find yourself using a nearest-neighbors type of approach, consider using a KD-Tree or a Ball Tree to speed it up.

There are good implementations of KD-Trees and nearest neighbours in scipy, sklearn and this [library](https://github.com/lmcinnes/pynndescent)

You can work your algorithm backwards (starting at the end and backing up to the starting point) or forwards (finding a starting point and looping until there is no time left). They will lead to different designs and results.

## Step 1: Imports

You can import the libraries you intend on using here or as you go along.

In [1]:
import numpy as np
import pandas as pd
import networkx as nx #visualize graphs from dataframes
import timeit
import math
import time

## Step 2: Load Data

The `bank_data.csv` file is located in the **data** folder. Make sure you use the correct path.

In [2]:
df = pd.read_csv('data/bank_data.csv')
df

,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393
...,...,...,...,...,...
9995,9995,-2.472127,4.787304,2100,0.494904
9996,9996,1.034968,-4.321240,54900,0.492866
9997,9997,-0.512720,-2.949173,32200,0.933033
9998,9998,2.626841,-3.482923,5100,0.551312


## Step 3: Data Exploration

Explore the data set to get a general understanding of what you're working with.

Feel free to add additional cells, and remove the ones you don't use.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10000 non-null  int64  
 1   x_coordinate  10000 non-null  float64
 2   y_coordinate  10000 non-null  float64
 3   money         10000 non-null  int64  
 4   time (hr)     10000 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 390.8 KB


In [4]:
df.describe()

,id,x_coordinate,y_coordinate,money,time (hr)
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,-0.016431,0.037561,18430.320000,0.750245
std,2886.89568,2.889568,2.881789,25382.131155,0.433541
min,0.00000,-4.999292,-4.999928,100.000000,0.000032
25%,2499.75000,-2.513854,-2.415033,900.000000,0.373141
50%,4999.50000,-0.024904,0.010442,5800.000000,0.749218
75%,7499.25000,2.461754,2.540303,26900.000000,1.122812
max,9999.00000,4.999851,4.999626,102300.000000,1.499851


In [5]:
df2 = df.head(5)
df2

,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393


In [6]:
import math
def calculateDistance(x1,y1,x2,y2):
    return math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 )
print(calculateDistance(2.851925,1.201712,4.150372,-2.654334))
print(timeit.timeit('[calculateDistance(2.851925,1.201712,4.150372,-2.654334)]', globals=globals()))

4.068790405750215
0.3352948590181768


In [7]:
def calculateDistance1(x1,y1,x2,y2):#chose this function because it is faster
    return math.hypot(x2 - x1, y2 - y1)
print(calculateDistance1(2.851925,1.201712,4.150372,-2.654334))
print(timeit.timeit('[calculateDistance1(2.851925,1.201712,4.150372,-2.654334)]', globals=globals()))

4.0687904057502156
0.22858237591572106


In [8]:
def travelTime(x1,y1,x2,y2):
    return (math.hypot(x2 - x1, y2 - y1))/30 #30km/h is the speed we are given
travelTime(2.851925,1.201712,4.150372,-2.654334)

0.13562634685834052

In [9]:
for x in df2:
    print(x)

id
x_coordinate
y_coordinate
money
time (hr)


In [10]:
for index,row in df2.iterrows():
    print(row['money'])

29700.0
6500.0
89400.0
96100.0
41100.0


In [11]:
df2

,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393


In [12]:
df2['money/time ratio']=df2['money']/df2['time (hr)']
df2

/tmp/ipykernel_35223/579337207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['money/time ratio']=df2['money']/df2['time (hr)']


,id,x_coordinate,y_coordinate,money,time (hr),money/time ratio
0,0,2.851925,1.201712,29700,0.273403,108630.921470
1,1,4.150372,-2.654334,6500,0.064040,101499.543603
2,2,-1.494092,-1.230419,89400,0.127458,701405.770444
3,3,1.271326,-0.088520,96100,1.315029,73078.228687
4,4,2.471113,-0.592810,41100,0.164393,250010.909373


In [13]:
#create a new column so we can figure out which have the best money/time to rob ratio
df.loc[:,'money/time ratio'] = df['money']/df['time (hr)']
df

,id,x_coordinate,y_coordinate,money,time (hr),money/time ratio
0,0,2.851925,1.201712,29700,0.273403,108630.921470
1,1,4.150372,-2.654334,6500,0.064040,101499.543603
2,2,-1.494092,-1.230419,89400,0.127458,701405.770444
3,3,1.271326,-0.088520,96100,1.315029,73078.228687
4,4,2.471113,-0.592810,41100,0.164393,250010.909373
...,...,...,...,...,...,...
9995,9995,-2.472127,4.787304,2100,0.494904,4243.243258
9996,9996,1.034968,-4.321240,54900,0.492866,111389.317396
9997,9997,-0.512720,-2.949173,32200,0.933033,34511.098257
9998,9998,2.626841,-3.482923,5100,0.551312,9250.652010


In [14]:
#sort 'money/time ratio' column by descending order so we can see which banks are the best to rob
sorted_df_descending = df.sort_values(by='money/time ratio',ascending=False)
sorted_df_descending

,id,x_coordinate,y_coordinate,money,time (hr),money/time ratio
3613,3613,-1.950527,-1.495858,54400,0.000186,2.929603e+08
9546,9546,-1.810721,-0.795421,75100,0.000788,9.530556e+07
3803,3803,3.867586,0.918332,18500,0.000358,5.169381e+07
6528,6528,-0.282937,3.224334,33900,0.000769,4.407762e+07
9583,9583,3.393835,-2.790830,24000,0.001131,2.121396e+07
...,...,...,...,...,...,...
1838,1838,-2.179862,-3.064824,100,1.486872,6.725528e+01
8832,8832,1.934915,-4.278272,100,1.490015,6.711343e+01
4764,4764,0.565196,1.437413,100,1.491462,6.704829e+01
2123,2123,-3.105083,-1.135305,100,1.494467,6.691347e+01


In [15]:
#reset our index values
sorted_df_descending = sorted_df_descending.reset_index(drop=True)
sorted_df_descending

,id,x_coordinate,y_coordinate,money,time (hr),money/time ratio
0,3613,-1.950527,-1.495858,54400,0.000186,2.929603e+08
1,9546,-1.810721,-0.795421,75100,0.000788,9.530556e+07
2,3803,3.867586,0.918332,18500,0.000358,5.169381e+07
3,6528,-0.282937,3.224334,33900,0.000769,4.407762e+07
4,9583,3.393835,-2.790830,24000,0.001131,2.121396e+07
...,...,...,...,...,...,...
9995,1838,-2.179862,-3.064824,100,1.486872,6.725528e+01
9996,8832,1.934915,-4.278272,100,1.490015,6.711343e+01
9997,4764,0.565196,1.437413,100,1.491462,6.704829e+01
9998,2123,-3.105083,-1.135305,100,1.494467,6.691347e+01


In [16]:
sorted5000 = sorted_df_descending.head(5000)
sorted5000

,id,x_coordinate,y_coordinate,money,time (hr),money/time ratio
0,3613,-1.950527,-1.495858,54400,0.000186,2.929603e+08
1,9546,-1.810721,-0.795421,75100,0.000788,9.530556e+07
2,3803,3.867586,0.918332,18500,0.000358,5.169381e+07
3,6528,-0.282937,3.224334,33900,0.000769,4.407762e+07
4,9583,3.393835,-2.790830,24000,0.001131,2.121396e+07
...,...,...,...,...,...,...
4995,6122,-2.655562,-0.629068,7400,0.741927,9.974030e+03
4996,3070,2.374337,0.607958,6700,0.671885,9.971944e+03
4997,7747,2.161209,2.811170,2100,0.210605,9.971256e+03
4998,691,4.723432,1.306989,14700,1.477027,9.952425e+03


In [17]:
#Please skip to Step 4 from here.

In [18]:
import time

start_time = time.time()
max_time = 24
total_money = 0
time_spent = 0
visited_banks = []

def travelTime(x1,y1,x2,y2):
    return (math.hypot(x2 - x1, y2 - y1))/30

for index,row in sorted5000.iterrows():

    total_money += row['money']
    time_spent += row['time (hr)']
    visited_banks.append(row['id'])#Append visited bank ID
    #visited_banks.sort()#sort the list of visited banks in ascending order of IDs
    
    if index < len(sorted5000):
        x1,y1 = row['x_coordinate'],row['y_coordinate']
        x2,y2 = sorted5000.loc[index+1,'x_coordinate'],sorted5000.loc[index+1,'y_coordinate']
        travel = travelTime(x1,y1,x2,y2)
        if time_spent + travel > max_time:
            break #Exit loop if time limit exceeded after travel
        time_spent += travel
        #print(f"Time before the exit: {time_spent}")
        
#if (time_spent < max_time) and (len(visited_banks) > 0):
    #grabs the last visited bank's row
last_bank = sorted5000[sorted5000['id'] == visited_banks[-1]]
    
    #grabs the coordinates of the last bank visited
last_x,last_y = last_bank['x_coordinate'].iloc[0],last_bank['y_coordinate'].iloc[0]
    
    #calculate the time it takes to travel from the last bank to the exit point
travel_to_exit = travelTime(last_x,last_y,0,0)
    
    #calculate the total amount of time for this endeavor
time_spent += travel_to_exit
    
    #give the number of banks visited
number_of_banks = len(visited_banks)
    
#visited_banks.append('Exit Point (0, 0)')
    
end_time = time.time()
elapsed_time = end_time-start_time


print(f"Total money collected: {total_money}")
print(f"Time spent robbing banks: {time_spent}")
print(f"Number of Banks visited: {number_of_banks}")
print(f"Banks visited: {visited_banks}")
print(f"Elapsed time: {elapsed_time} seconds")

Total money collected: 6466700.0
Time spent robbing banks: 24.058835697407567
Number of Banks visited: 129
Banks visited: [3613.0, 9546.0, 3803.0, 6528.0, 9583.0, 5933.0, 9195.0, 4757.0, 8550.0, 4499.0, 9241.0, 7343.0, 3798.0, 7544.0, 4762.0, 8436.0, 8562.0, 6097.0, 487.0, 2928.0, 5135.0, 865.0, 3914.0, 4987.0, 3466.0, 7560.0, 8579.0, 8690.0, 8469.0, 433.0, 524.0, 6468.0, 1914.0, 1757.0, 8287.0, 7258.0, 4725.0, 8206.0, 4605.0, 7064.0, 3926.0, 9401.0, 9736.0, 2331.0, 9653.0, 7074.0, 2346.0, 8966.0, 4696.0, 8022.0, 4789.0, 6740.0, 9881.0, 70.0, 2656.0, 951.0, 209.0, 58.0, 9228.0, 3193.0, 8525.0, 5610.0, 2037.0, 4287.0, 6022.0, 2827.0, 8849.0, 9378.0, 6104.0, 2741.0, 7701.0, 8355.0, 5563.0, 3005.0, 7665.0, 6987.0, 5719.0, 2729.0, 1997.0, 8286.0, 4906.0, 613.0, 790.0, 7087.0, 5126.0, 5562.0, 3516.0, 2243.0, 6759.0, 3026.0, 1372.0, 2442.0, 1733.0, 8375.0, 7764.0, 7649.0, 7689.0, 9880.0, 3297.0, 4794.0, 3089.0, 7583.0, 5725.0, 517.0, 4345.0, 2521.0, 2190.0, 5155.0, 9049.0, 444.0, 6712.0, 890

## Step 4: Solution 

Your solution must provide:

- A list of bank IDs in order that you rob them
- Total amount you were able to rob
- Time it took to rob + escape
- The execution time for your code to run

**Important: Your solution must respect all the constraints mentioned in the description.**

Feel free to add additional cells, and remove the ones you don't use.

In [19]:
import time

start_time1 = time.time()
max_time = 24
total_money = 0
time_spent = 0
visited_banks = []

def travelTime(x1,y1,x2,y2):
    return (math.hypot(x2 - x1, y2 - y1))/30

#Iterate through the banks 
for index,row in sorted5000.iterrows():

    total_money += row['money']#Add up the money that is in each bank
    time_spent += row['time (hr)']#Add up the time that is spent at each bank
    visited_banks.append(row['id'])#Append visited bank ID
    #visited_banks.sort()#sort the list of visited banks in ascending order of IDs
    
    if index < len(sorted5000):
        x1,y1 = row['x_coordinate'],row['y_coordinate']
        x2,y2 = sorted5000.loc[index+1,'x_coordinate'],sorted5000.loc[index+1,'y_coordinate']
        travel = travelTime(x1,y1,x2,y2)#Calculate the time it takes to travel in between each bank
        if time_spent + travel > max_time:
            break #Exit loop if time limit exceeded after travel
        time_spent += travel #Add up the travel time and time it takes to rob each bank
        
        

#grabs the last visited bank's row
last_bank = (sorted5000[sorted5000['id'] == visited_banks[-1]])
    
#grabs the coordinates of the last bank visited
last_x,last_y = last_bank['x_coordinate'].iloc[0],last_bank['y_coordinate'].iloc[0]
    
#calculate the time it takes to travel from the last bank to the exit point
travel_to_exit = travelTime(last_x,last_y,0,0)
    
#calculate the total amount of time for this endeavor
time_spent += travel_to_exit
    
#give the number of banks visited
number_of_banks = len(visited_banks)
    
#Calculates the amount of time it takes for the solution to run
end_time1 = time.time()
elapsed_time1 = end_time1-start_time1


print(f"Total money collected: {total_money}")
print(f"Time spent robbing banks: {time_spent}")
print(f"Number of Banks visited: {number_of_banks}")
print(f"Banks visited: {visited_banks}")
print(f"Elapsed time: {elapsed_time1} seconds")

Total money collected: 6466700.0
Time spent robbing banks: 24.058835697407567
Number of Banks visited: 129
Banks visited: [3613.0, 9546.0, 3803.0, 6528.0, 9583.0, 5933.0, 9195.0, 4757.0, 8550.0, 4499.0, 9241.0, 7343.0, 3798.0, 7544.0, 4762.0, 8436.0, 8562.0, 6097.0, 487.0, 2928.0, 5135.0, 865.0, 3914.0, 4987.0, 3466.0, 7560.0, 8579.0, 8690.0, 8469.0, 433.0, 524.0, 6468.0, 1914.0, 1757.0, 8287.0, 7258.0, 4725.0, 8206.0, 4605.0, 7064.0, 3926.0, 9401.0, 9736.0, 2331.0, 9653.0, 7074.0, 2346.0, 8966.0, 4696.0, 8022.0, 4789.0, 6740.0, 9881.0, 70.0, 2656.0, 951.0, 209.0, 58.0, 9228.0, 3193.0, 8525.0, 5610.0, 2037.0, 4287.0, 6022.0, 2827.0, 8849.0, 9378.0, 6104.0, 2741.0, 7701.0, 8355.0, 5563.0, 3005.0, 7665.0, 6987.0, 5719.0, 2729.0, 1997.0, 8286.0, 4906.0, 613.0, 790.0, 7087.0, 5126.0, 5562.0, 3516.0, 2243.0, 6759.0, 3026.0, 1372.0, 2442.0, 1733.0, 8375.0, 7764.0, 7649.0, 7689.0, 9880.0, 3297.0, 4794.0, 3089.0, 7583.0, 5725.0, 517.0, 4345.0, 2521.0, 2190.0, 5155.0, 9049.0, 444.0, 6712.0, 890

In [20]:
#Below is the code for when I changed the max_time to 23.8 hours so I could run to the helipad within the 24 hours required.

#Change max_time to 23.8
import time

start_time = time.time()
max_time = 23.8
total_money = 0
time_spent = 0
visited_banks = []

def travelTime(x1,y1,x2,y2):
    return (math.hypot(x2 - x1, y2 - y1))/30

for index,row in sorted5000.iterrows():

    total_money += row['money']
    time_spent += row['time (hr)']
    visited_banks.append(row['id'])#Append visited bank ID
    #visited_banks.sort()#sort the list of visited banks in ascending order of IDs
    
    if index < len(sorted5000):
        x1,y1 = row['x_coordinate'],row['y_coordinate']
        x2,y2 = sorted5000.loc[index+1,'x_coordinate'],sorted5000.loc[index+1,'y_coordinate']
        travel = travelTime(x1,y1,x2,y2)
        if time_spent + travel > max_time:
            break #Exit loop if time limit exceeded after travel
        time_spent += travel
        #print(f"Time before the exit: {time_spent}")
        
#if (time_spent < max_time) and (len(visited_banks) > 0):
    #grabs the last visited bank's row
last_bank = sorted5000[sorted5000['id'] == visited_banks[-1]]
    
    #grabs the coordinates of the last bank visited
last_x,last_y = last_bank['x_coordinate'].iloc[0],last_bank['y_coordinate'].iloc[0]
    
    #calculate the time it takes to travel from the last bank to the exit point
travel_to_exit = travelTime(last_x,last_y,0,0)
    
    #calculate the total amount of time for this endeavor
time_spent += travel_to_exit
    
    #give the number of banks visited
number_of_banks = len(visited_banks)
    
#visited_banks.append('Exit Point (0, 0)')
    
end_time = time.time()
elapsed_time = end_time-start_time


print(f"Total money collected: {total_money}")
print(f"Time spent robbing banks: {time_spent}")
print(f"Number of Banks visited: {number_of_banks}")
print(f"Banks visited: {visited_banks}")
print(f"Elapsed time: {elapsed_time} seconds")

### Briefly explain your solution/approach, and how it works.

1)Create a new column with the ratio of bank money versus time it takes to rob the bank in order to determine which banks would be the best to rob.

2)Sort our dataframe in descending order based on this new ratio.

3)Iterate through the rows of the dataframe and calculate the money you accumulate as you rob each bank. In the meantime you add up the amount the time it takes to rob each bank and the time it takes to travel in between each bank.

4)The loop stops once the time accumulated surpasses the max time allowed.

5)Unfortunately I was unable to figure how to incorporate the exit point. Basically once my loop had stopped and I finished robbing the last bank I travelled to my exit time. As a result, my robber gets caught. 

6)The only I could figure out how to circumvate this is if I manually modify my max_time. You can see when I modify my max_time to 23.8 the robber is able to get out under 24 hours. The block of code where I modified the time to 23.8 is in a markdown box right below the code where the max_time is 24 hours if you wish to run it it to see.

## Step 5: Summary and Results

1. What is the total amount of money you were able to collect?


In [21]:
print(f"Total money collected: {total_money}")

Total money collected: 6466700.0


2. What are the IDs of the banks you robbed, in order? Use the list from your solution.

In [22]:
print(f"Banks visited: {visited_banks}")

Banks visited: [3613.0, 9546.0, 3803.0, 6528.0, 9583.0, 5933.0, 9195.0, 4757.0, 8550.0, 4499.0, 9241.0, 7343.0, 3798.0, 7544.0, 4762.0, 8436.0, 8562.0, 6097.0, 487.0, 2928.0, 5135.0, 865.0, 3914.0, 4987.0, 3466.0, 7560.0, 8579.0, 8690.0, 8469.0, 433.0, 524.0, 6468.0, 1914.0, 1757.0, 8287.0, 7258.0, 4725.0, 8206.0, 4605.0, 7064.0, 3926.0, 9401.0, 9736.0, 2331.0, 9653.0, 7074.0, 2346.0, 8966.0, 4696.0, 8022.0, 4789.0, 6740.0, 9881.0, 70.0, 2656.0, 951.0, 209.0, 58.0, 9228.0, 3193.0, 8525.0, 5610.0, 2037.0, 4287.0, 6022.0, 2827.0, 8849.0, 9378.0, 6104.0, 2741.0, 7701.0, 8355.0, 5563.0, 3005.0, 7665.0, 6987.0, 5719.0, 2729.0, 1997.0, 8286.0, 4906.0, 613.0, 790.0, 7087.0, 5126.0, 5562.0, 3516.0, 2243.0, 6759.0, 3026.0, 1372.0, 2442.0, 1733.0, 8375.0, 7764.0, 7649.0, 7689.0, 9880.0, 3297.0, 4794.0, 3089.0, 7583.0, 5725.0, 517.0, 4345.0, 2521.0, 2190.0, 5155.0, 9049.0, 444.0, 6712.0, 8908.0, 8703.0, 7877.0, 7595.0, 8231.0, 9275.0, 4465.0, 5295.0, 5296.0, 299.0, 8846.0, 6254.0, 507.0, 6535.0, 

3. How much time did it take to rob all the banks + escape? Does your solution meet the 24 hour constraint? 

In [23]:
print(f"Time spent robbing banks: {time_spent}")
#unfortunately did not meet the 24 hours constraint

Time spent robbing banks: 24.058835697407567


4. What is the execution time of your solution? Does your solution run in 3 minutes or less?

In [24]:
print(f"Elapsed time: {elapsed_time1} seconds")

Elapsed time: 0.016382932662963867 seconds


In [ ]:
from check_solution import distance, check_solution
check_solution(lst, df)

lst = the list that has the IDs of your selected banks (in the correct order)
df = the df where you loaded the csv file